In [2]:
# Setup

import os
import sys
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.advanced_activations import Quorum
from keras.datasets import cifar100
from keras.utils import np_utils
from keras import backend as K
from keras.activations import relu
from keras.callbacks import ModelCheckpoint

af = "pap" #pap, relu, thresh, notrain
init_fn = "he_normal"

def pushln(line):
    sys.stdout.write(line)
    sys.stdout.flush()
    
def step(X):
    return K.switch(X <= 0, 0, 1)

def prepare_input_data(X_train, X_test):
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0
    return X_train, X_test

def prepare_output_data(y_train, y_test):
    y_train = np_utils.to_categorical(y_train)
    y_test = np_utils.to_categorical(y_test)
    return y_train, y_test

def get_af(name):
    af = name.lower()
    
    if af == "relu":
        return Activation('relu')
    elif af == "pap":
        return Quorum([relu, step])
    elif af == "thresh":
        return Quorum([relu, step], threshold=0.5)
    elif af == "notrain":
        return Quorum([relu, step], trainable=False)
    else:
        raise RuntimeError("Unrecognized activation function: {}".format(name))

def add_convolutional_layers(model, activation_name, n, filter_size, window, 
                             dropout=0.25, stack_finished=True, input_shape=None):
    if input_shape:
        model.add(Convolution2D(filter_size, window, window, border_mode="same", 
				init=init_fn, input_shape=input_shape))
        model.add(get_af(af))
        n=n-1
        
    for i in range(n):
        model.add(Convolution2D(filter_size, window, window, border_mode="same", init=init_fn))
        model.add(get_af(activation_name))
        
    if stack_finished:
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(dropout))
        
# Script

(X_train, y_train), (X_test, y_test) = cifar100.load_data()
X_train, X_test = prepare_input_data(X_train, X_test)
y_train, y_test = prepare_output_data(y_train, y_test)

model = Sequential()

# 2 x 32 x 3
add_convolutional_layers(model, af, 2, 32, 3, input_shape=(3, 32, 32))

# 2 x 64 x 3
add_convolutional_layers(model, af, 2, 64, 3)

# 1 x 100 x FC
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(get_af(af))
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Activation('softmax'))

pushln("Compiling model...")
model.compile(loss='categorical_crossentropy', optimizer='sgd')
pushln("finished.\n")

datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=True,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

pushln("Fitting ImageDataGenerator...")
datagen.fit(X_train)
pushln("finished...\n")
mc = ModelCheckpoint("weights_"+af, verbose=1)
pushln("Beginning training...\n")
hist = model.fit_generator(datagen.flow(X_train, y_train, batch_size=128), verbose=1,
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=200, show_accuracy=True,
                        validation_data=(X_test, y_test),
                        nb_worker=1, callbacks=[mc])


169009152/169001437 [==============================] - 28s    
Untaring file...
Compiling model...finished.
Fitting ImageDataGenerator...